# Loading the dataset and preprocessing the data:

In [ ]:
from datasets import load_dataset

ds = load_dataset("eswardivi/medical_qa")
ds = ds.remove_columns(["input"]).rename_column("instruction", "input")
train_test_split = ds["train"].train_test_split(test_size=307, train_size=6000, shuffle=True, seed=42)
new_ds = {"train": train_test_split["train"], "test": train_test_split["test"]}

# Verify the changes
print(new_ds)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

data_merged.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/6307 [00:00<?, ? examples/s]

{'train': Dataset({
    features: ['input', 'output'],
    num_rows: 6000
}), 'test': Dataset({
    features: ['input', 'output'],
    num_rows: 307
})}


In [ ]:
new_ds['train'][2]

{'input': "I'm aware that people are recommended to stay away from taking ibuprofen, cortisone, and other anti-inflammatory drugs. But does this also apply to supplements? For example, St. John's Wort, green tea, and turmeric are all supposed to have anti-inflammatory properties. Would it be better to avoid or limit their consumption at this time?",
 'output': "Most health care groups have not made any recommendations regarding ibuprofen or other NSAIDs. We simply don't have the information to be able to recommend one way or another. If you have been taking these supplements, it should be fine to continue taking them."}

# Initializing the Qwen 2.5-0.5B model and tokenizer:

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "Qwen/Qwen2.5-0.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",              # GPU if available
    dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    trust_remote_code=True
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/681 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

In [ ]:
pip install transformers datasets peft trl bitsandbytes accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 9.1 MB/s eta 0:00:00


In [ ]:
from transformers import TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer

# Defining format prompt funciton:

In [ ]:
def prompt_instruction_format(sample):
    return f"""### Instruction:
Use the input below to provide a concise and accurate medical answer.

### Input:
{sample['input']}

### Response:
{sample['output']} <|END_OF_TEXT|>"""

# Finetuning the model using LoRA:

In [ ]:
peft_config = LoraConfig(
    r=16,  # LoRA rank (lower for small models to prevent overfitting)
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.05,  # Dropout for regularization
    bias="none",  # No bias adaptation
    task_type="CAUSAL_LM",  # For causal language modeling
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]  # Qwen-specific modules
)

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
from trl import SFTTrainer, SFTConfig

sft_args = SFTConfig(
    output_dir="./output",
    num_train_epochs=1,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    logging_steps=10,
    eval_strategy="no",
    save_strategy="steps",
    save_steps=100,
    packing=False,
    max_length=512,
    report_to="none",
)

trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,   # pass tokenizer here on new TRL
    train_dataset=new_ds["train"],
    eval_dataset=new_ds["test"],
    peft_config=peft_config,
    formatting_func=prompt_instruction_format,  # must return List[str]
    args=sft_args,
)


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Applying formatting function to train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/6000 [00:00<?, ? examples/s]

Applying formatting function to eval dataset:   0%|          | 0/307 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/307 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/307 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/307 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

# Step 8: Save the fine-tuned model and tokenizer
model.save_pretrained("./fine_tuned_qwen_medical_qa")
tokenizer.save_pretrained("./fine_tuned_qwen_medical_qa")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Step,Training Loss
10,2.823600
20,2.441000
30,2.190900
40,2.284800
50,2.056400
60,2.112100
70,2.220000
80,2.020000
90,2.155600
100,2.176400


('./fine_tuned_qwen_medical_qa/tokenizer_config.json',
 './fine_tuned_qwen_medical_qa/special_tokens_map.json',
 './fine_tuned_qwen_medical_qa/chat_template.jinja',
 './fine_tuned_qwen_medical_qa/vocab.json',
 './fine_tuned_qwen_medical_qa/merges.txt',
 './fine_tuned_qwen_medical_qa/added_tokens.json',
 './fine_tuned_qwen_medical_qa/tokenizer.json')

# Testing the model:

In [ ]:
test_sample = new_ds["test"][0]
prompt = prompt_instruction_format(test_sample)
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
Caching is incompatible with gradient checkpointing in Qwen2DecoderLayer. Setting `past_key_values=None`.
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


### Instruction:
Use the input below to provide a concise and accurate medical answer.

### Input:
Hello, sir, your jaw is in pain, especially when you chew or yawn.

### Response:
In order to confirm the diagnosis, you need to undergo a diagnostic procedure such as an ophthalmology, evaluation, diagnosis, and ophthalmology, as well as a diagnostic procedure for ophthalmology, treatment, nose, mouth, and pharynx, which may require a central line of veins to be inserted to help diagnose and treat the disorder. <|END_OF_TEXT|>


In [ ]:
from google.colab import drive
drive.mount('/content/drive')